In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
website = "https://bina.az/alqi-satqi"
path = "/Users/azerm/Dropbox/My PC (LAPTOP-Q3RVDDOQ)/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

In [26]:
options = Options()
options.headless = False
service = Service(executable_path = path)
driver = webdriver.Chrome(service=service,options=options)
driver.get(website)
driver.maximize_window()

<ipython-input-26-d9316773a4f1>:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


In [27]:
def get_urls(items):
    item_urls = []
    for item in items:
        apartment_link = item.find_element(by="xpath",value="./a[@class='item_link']")
        link_full = apartment_link.get_attribute("href")
        item_urls.append(link_full)
    return item_urls
    


item_urls = []
items = driver.find_elements(by="xpath",value='//div[@class="items_list"]/div[contains(@class, "items-i")]')
new_urls = get_urls(items)
item_urls.extend(new_urls)
while(len(items)%24==0 and len(item_urls)<240):
    next_btn = driver.find_element(by="xpath",value='//span[@class="next"]')
    next_btn.click()
    items = driver.find_elements(by="xpath",value='//div[@class="items_list"]/div[contains(@class, "items-i")]')
    new_urls = get_urls(items)
    item_urls.extend(new_urls)

In [28]:
len(item_urls)

240

In [29]:
item_urls[-1]

'https://bina.az/items/2979569'

In [56]:
import time
def scrape_description_data(item_url):
    
    try:
    # Wait for the ad or popup to appear (adjust the timeout as needed)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//span[@class="close-btn1"]')))
    
    # Locate the close button or element of the ad
        close_button = driver.find_element(By.XPATH, '//span[@class="close-btn1"]')

        # Click the close button to close the ad
        close_button.click()
        driver.switch_to.default_content()

    except Exception as e:
        # Handle exceptions (e.g., ad not found or close button not found)
        print("continue")
        
    
    apartment_dict = {}
    apartment_dict["url"] = item_url
    driver.get(item_url)
    #title
    title = driver.find_element(by="xpath",value="//h1[@class='product-title']").text
    apartment_dict["title"] = title
    
    #properties
    properties = driver.find_elements(by="xpath",value="//div[@class='product-properties__column']/div[@class='product-properties__i']")
    properties_dict = {}
    for prop in properties:
        key = prop.find_element(by="xpath",value=".//label[@class='product-properties__i-name']").text
        value = prop.find_element(by="xpath",value=".//span[@class='product-properties__i-value']").text
        apartment_dict[key] = value
        properties_dict[key] = value
    
    #description
    description_elements = driver.find_elements(by="xpath",value=".//div[@class='product-description__content']/p")
    description = ''
    for desc in description_elements:
        description+=desc.text
    apartment_dict["description"] = description
    
    #price
    price_element = driver.find_element(by="xpath",value=".//div[@class='product-price']/div[@class='product-price__i product-price__i--bold']")
    price_value = price_element.find_element(by="xpath",value=".//span[@class='price-val']").text
    price_currency = price_element.find_element(by="xpath",value=".//span[@class='price-cur']").text
    apartment_dict["price_value"] = price_value
    apartment_dict["price_currency"] = price_currency
    
    #contact
    
    agent = driver.find_element(by="xpath",value=".//div[@class='product-owner__info-name']").text
    apartment_dict["agent"] = agent
    
    show_number_btn = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ".//div[@class='product-phones bz-mt-15']"))
    )
    show_number_btn.click()
    
    
    contact_number = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="product-phones__list-i"]/a'))
    )
    apartment_dict["contact_number"] = contact_number.text
    
    #unvan
    unvan = driver.find_element(by="xpath",value=".//div[@class='product-map__left__address']").text
    apartment_dict["unvan"] = unvan
    locations = []
    locations_elements = driver.find_elements(by="xpath",value=".//ul[@class='product-extras bz-d-flex bz-align-center bz-gap-15 bz-wrap-wrap']/li[@class='product-extras__i']")
    for loc in locations_elements:
        locations.append(loc.find_element(by="xpath",value=".//a").text)
    apartment_dict["locations"] = locations

    open_map = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='product-map__right open_map']"))
    )                        
    open_map.click()
    time.sleep(5)
    map_data = driver.find_element(by="xpath",value="//div[@class='reveal-modal reveal-modal_opened']/div[@class='reveal-content']/div[@id='item_map']")
    latitude = map_data.get_attribute("data-lat")
    longitude = map_data.get_attribute("data-lng")
    apartment_dict["latitude"] = latitude
    apartment_dict["longitude"] = longitude
    return apartment_dict
    
    

In [57]:
data = []
for item_url in item_urls[:10]:
    alma = scrape_description_data(item_url)
    data.append(alma)
data

continue
continue
continue
continue
continue
continue
continue
continue
continue
continue


[{'url': 'https://bina.az/items/3753854',
  'title': 'Satılır 2 otaqlı yeni tikili 82 m², Azadlıq Prospekti m.',
  'Kateqoriya': 'Yeni tikili',
  'Mərtəbə': '8 / 17',
  'Sahə': '82 m²',
  'Otaq sayı': '2',
  'Çıxarış': 'var',
  'description': 'VASİTEÇİLER VE AGENTLIKLER NARAHAT ETMESIN!\nOBA MTK-8ci mkr,lukoil yanacaq doldurma məntəqəsinə yaxın,qrand marketden irelide yerlesir.\nMenzil qanuni 2-otaqdan ibarətdir,orta temirlidir ve istilik sistemi kombidir.\nTam esyali satilir.\nSenedleri qaydasindadir cixaris(kupcadir) ve ipotekaya yararlıdır.\nEN son qiymeti 165.000 azn.Ofis haqqi 1%.',
  'price_value': '165 000',
  'price_currency': 'AZN',
  'agent': 'Aga',
  'contact_number': '(070) 561-52-96',
  'unvan': 'Cəfər Xəndan küç.',
  'locations': ['Azadlıq Prospekti m.', 'Binəqədi r.', '8-ci mikrorayon q.'],
  'latitude': '40.4209144',
  'longitude': '49.842779500000006'},
 {'url': 'https://bina.az/items/3704166',
  'title': 'Satılır 3 otaqlı köhnə tikili 85 m², Nəriman Nərimanov m.',
  '

In [58]:
df = pd.DataFrame(data)

In [59]:
df

,url,title,Kateqoriya,Mərtəbə,Sahə,Otaq sayı,Çıxarış,description,price_value,price_currency,agent,contact_number,unvan,locations,latitude,longitude,Təmir,Torpaq sahəsi,İpoteka
0,https://bina.az/items/3753854,"Satılır 2 otaqlı yeni tikili 82 m², Azadlıq Pr...",Yeni tikili,8 / 17,82 m²,2,var,VASİTEÇİLER VE AGENTLIKLER NARAHAT ETMESIN!\nO...,165 000,AZN,Aga,(070) 561-52-96,Cəfər Xəndan küç.,"[Azadlıq Prospekti m., Binəqədi r., 8-ci mikro...",40.4209144,49.842779500000006,NaN,NaN,NaN
1,https://bina.az/items/3704166,"Satılır 3 otaqlı köhnə tikili 85 m², Nəriman N...",Köhnə tikili,2 / 4,85 m²,3,var,"Təcili!!! Yeni təmirli!!\nNərimanov rayonu, Mo...",200 000,AZN,Xalid,(010) 255-95-85,Ismayil Hidayətzadə küç,"[Rəssamlıq Akademiyası, Nəriman Nərimanov park...",40.39686541390212,49.87223488849583,var,NaN,NaN
2,https://bina.az/items/3402747,"Satılır həyət evi / bağ evi 46 m², Şüvəlan q.",Həyət evi / Bağ evi,NaN,46 m²,NaN,var,Təcili satılır !\n2 sot torpaq sahəsinin içind...,77 000,AZN,Bəxtiyar,(050) 444-25-96,Mayak yolu,"[Xəzər r., Şüvəlan q.]",40.497,50.208,var,2 sot,NaN
3,https://bina.az/items/3771265,"Satılır 3 otaqlı yeni tikili 91 m², Xırdalan",Yeni tikili,5 / 15,91 m²,3,yoxdur,Xirdalanin Tam Merkezinde Teze Bazarin yaninda...,105 000,AZN,Daşqın,(055) 830-70-36,Heydər Əliyev pr.,[],40.45206984615483,49.73282937306521,var,NaN,NaN
4,https://bina.az/items/3771263,"Satılır 3 otaqlı yeni tikili 123 m², Nəriman N...",Yeni tikili,8 / 19,123 m²,3,var,Nerimanov metrosunun yaxinliqinda yerləşən 19 ...,285 000,AZN,Əbülfəz,(050) 209-37-56,Əliyar Əliyev küç.,"[Nəriman Nərimanov parkı, Nəriman Nərimanov m....",40.40175987995332,49.87198185402826,var,NaN,NaN
5,https://bina.az/items/3771260,"Satılır torpaq 21 sot, Sulutəpə q.",Torpaq,NaN,21 sot,NaN,yoxdur,Sulutepe qəsəbəsi yol kənarında hər bir obyekt...,315 000,AZN,İSAXAN,(077) 719-13-03,Sulutəpə qəs.,"[Binəqədi r., Sulutəpə q.]",40.4367621,49.768319099999985,NaN,NaN,NaN
6,https://bina.az/items/3765678,"Satılır obyekt 45 m², Nəriman Nərimanov m.",Obyekt,NaN,45 m²,NaN,var,SAHİBİNDƏNNərimanov rayonu Ⓜ️etronun cıxışında...,450 000,AZN,Ağalar m.,(077) 380-00-14,M. Məmmədzadə küçəsi,"[Nəriman Nərimanov parkı, Nəriman Nərimanov m....",40.40244617376457,49.87532925087884,var,NaN,var
7,https://bina.az/items/3088052,"Satılır 4 otaqlı həyət evi / bağ evi 200 m², M...",Həyət evi / Bağ evi,NaN,200 m²,4,var,"Xəzər rayonu Mərdəkan qəsəbəsi ""Bravo"" marketə...",280 000,AZN,Murad,(055) 738-11-96,Mərdəkan q.,"[Xəzər r., Mərdəkan q.]",40.48061610869761,50.14598322529635,var,6 sot,NaN
8,https://bina.az/items/3705411,"Satılır 4 otaqlı həyət evi / bağ evi 150 m², M...",Həyət evi / Bağ evi,NaN,150 m²,4,var,Ekoloji cehetden temiz havasi olan Merdekenda ...,205 000,AZN,Kəmran,(070) 887-61-45,Merdekan qesebesi 5.,"[Xəzər r., Mərdəkan q.]",40.48391745099677,50.12864104769894,var,5 sot,NaN
9,https://bina.az/items/3755355,"Satılır 6 otaqlı həyət evi / bağ evi 500 m², M...",Həyət evi / Bağ evi,NaN,500 m²,6,var,Ekoloji cehetden temiz havasi olan Merdekenda1...,950 000,AZN,Kəmran,(070) 887-61-45,Mərdəkan qəsəbəsi,"[Xəzər r., Mərdəkan q.]",40.48861757685779,50.13482085726925,var,12 sot,NaN


In [55]:
scrape_description_data("https://bina.az/items/3728348")

continue


{'url': 'https://bina.az/items/3728348',
 'title': 'Satılır 2 otaqlı yeni tikili 58 m², Xırdalan',
 'Kateqoriya': 'Yeni tikili',
 'Mərtəbə': '5 / 9',
 'Sahə': '58 m²',
 'Otaq sayı': '2',
 'Çıxarış': 'var',
 'Təmir': 'var',
 'description': 'Ünvan : Xırdalan Şəhəri Tam Mərkəzdə Kristal Abşeron-1 Yaşayış Kompleksinde bazara, mektebe yaxin.Menzilde Kombi sistemi ve s. her bir şerait movcuddu.\nTam Əşyali\nNöv : Yeni tikili\nMərtəbə : 9/5\nOtaq : 2 Otağ\nSahə : 58 km²\nTəmir : Əla\nSənəd : Kupçali\nQaz, Su , işıq daimidir.💰Ofis Haqqi 1%. Etrafli melumat ucun Elaqe saxlayin.\n✅Qeyd: Xırdalanda az hallarda bu qiymətə belə təmirli mənzil çıxır, Bina tam yaşayışlıdı almaq məsləhətdi.',
 'price_value': '87 000',
 'price_currency': 'AZN',
 'agent': 'Müşviq',
 'contact_number': '(070) 338-08-01',
 'unvan': 'H.Əliyev pr.',
 'locations': [],
 'latitude': '40.45201745818469',
 'longitude': '49.74317080345372'}